All imports

In [55]:
import os
import json
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

* Remember to delet the nltk.download('all') after the download is complete it is only required to run once.

In [56]:
# Delet this cell after running once

import nltk
#nltk.download('all')

Loading the all the reviews in json file

In [57]:
# Loading current directory

current_directory = os.getcwd()

# Loading the reviews.json file

with open(fr'{current_directory}\Data collection/reviews.json', encoding= 'utf-8') as f:
    data = json.load(f)

# Iterating through each item in the json file and storing them as dictionary of Movie : Review pairs in a list
reviews_list = [{'Movie': movie, 'Review': review} for movie, review in data.items()]

# Createing DataFrame from list of dictionaries
df = pd.DataFrame(reviews_list)

Preprocessing text

In [58]:
# Loading stopwords

stop_words = stopwords.words('english')

def preprocessed_txt(text, stop_word):

    # Tokenizing the text.

    tokens = word_tokenize(text)

    # Removing stopwords.

    filtered_tokens = [token for token in tokens if token not in stop_word]

    # Lemmetizing the tokens.

    lemmetizer = WordNetLemmatizer()
    lemmetized_token = [lemmetizer.lemmatize(token) for token in filtered_tokens]

    # Joining the lemmetized tokens into string.

    processed_text = ' '.join(lemmetized_token)

    return processed_text

Preprocessing using batch processing

In [59]:
# Initializing the batch size.

batch_size = 50
total_reviews = len(df)

# Applying the processed_txt function to each row of review column in the data frame in batch of 50 iterations for memory efficiency

for i in range(0, total_reviews, batch_size):
    df.loc[i:i+batch_size, 'Review'] = df.loc[i:i+batch_size, 'Review'].apply(preprocessed_txt, args=(stop_words,))

Loading all positive and negative words

In [60]:
# Loading the positive and negative words excel file

PN_path = fr'{current_directory}/Positive and Negative Word List.xlsx'       # PN is abrivation for Positive Negative
PN_df = pd.read_excel(PN_path)

# Initializing negative words and positive words inside lists respectively

negative_words = PN_df['Negative Sense Word List'].tolist()
positive_words = PN_df['Positive Sense Word List'].tolist()

Calculating poitive and negative scores

* Numpy's np.isin() function is used to calculate the positive sore and negative score to speed up the process. np.isin() returns boolean array which is efficeant alternative to directly comparing the words potentially vectorizing the tokens and positive and negative words.

In [61]:
# Define functions to calculate positive, negative, subjectivity and polarity scores

def P_score(review):
    tokens = word_tokenize(review.lower())
    positive_score = np.sum(np.isin(tokens, positive_words))
    return positive_score

def N_score(review):
    tokens = word_tokenize(review.lower())
    negative_score = np.sum(np.isin(tokens, negative_words))
    return negative_score

def sub_score(review):
    tokens = word_tokenize(review.lower())
    subjectivity_score = ((P_score(review) + N_score(review)) /(len(tokens) + 0.000001))
    return subjectivity_score

# Apply the functions to each review in the 'Review' column using vectorization

df['positive_score'] = df['Review'].apply(P_score)
df['negative_score'] = df['Review'].apply(N_score)
df['polarity_score'] = df.apply(lambda row: (row['positive_score'] - row['negative_score'])/(row['positive_score'] + row['negative_score'] + 0.000001), axis=1)
df['subjectivity_score'] = df['Review'].apply(sub_score)
df.drop('Review', axis=1, inplace=True)

Saving the file

In [62]:
save_path = fr'{current_directory}/Sentiment_Analysis.xlsx'
df.to_excel(save_path)
print('File saved')
df

File saved


,Movie,positive_score,negative_score,polarity_score,subjectivity_score
0,"10,000 BC",825,293,0.475850,0.278317
1,101 Dalmatians,674,185,0.569267,0.280261
2,13 Going on 30,715,177,0.603139,0.285897
3,1408,904,443,0.342242,0.256279
4,17 Again,1097,233,0.649624,0.280709
...,...,...,...,...,...
736,Zodiac,465,171,0.462264,0.280176
737,Zombieland,343,110,0.514349,0.285985
738,Zwartboek,634,197,0.525872,0.266688
739,[Rec],687,268,0.438743,0.291247
